The next cell import all the libraries and the other folder that we will need

In [8]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [9]:
import cv2

In [10]:
import math

import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import size
from matplotlib import colors
from asgiref.sync import sync_to_async #in order to load variables to communicate between sync and async

import optimal_path as op

from robot import Robot
from Map import Map
from Local_navigation import* 
from Mouvement import*
from vision import*


initialize the initial grid and position

In [11]:
#start , goal , grid = "init_map"
start_pos = (0,0)
goal = (43,33)
map_lenght = 5
nb_of_square_by_side = 50
global motor_left_target, motor_right_target



# Useful functions

In [12]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    global motor_left_target, motor_right_target
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed

In [13]:
@tdmclient.notebook.sync_var
def proxi():
    """
    Returns the proximity values of the Thymio 
    """
    global prox_horizontal
    return prox_horizontal

Next cell is to create an instance Robot and Map 
  

In [11]:
George = Robot(start_pos, goal)
Lausanne = Map(map_lenght, nb_of_square_by_side)

Global control 

--> Optimal path calculation

In [7]:
# Compute the optimal path and visited nodes
path, visitedNodes = op.path_computation(George.get_start() , George.get_goal() , Lausanne.get_lenght(), Lausanne.get_map())

George.set_path(path) 
George.set_visit_nodes(visitedNodes) 

# Display the optimal path
op.display_map(Lausanne.get_lenght(),  Lausanne.get_map(),  George.get_visit_nodes(), George.get_path(), George.get_start(), George.get_goal())


NameError: name 'George' is not defined

start threads

In [ ]:
# Threads de l'acquisition d'image et filtre Kalman --> C'est l'ordi qui envoit ça.
# Met à jour la position du robot
# Affiche sur ordi la map analisée

# Thread pour les roues? --> implique class motors

# Main loop

In [ ]:
global move 
move = False #bool which indicate how our robot is moving (1: avoid an object VS 0: following the optimal path)
stop_video = False

#VideoCap = vision_initialization()
VideoCap=cv2.VideoCapture(0)
print('Hello World')
ret, frame=VideoCap.read()

gr_points,  gr_mask, gr_contours=detect_inrange(frame, 10000, green)
if (len(gr_points)>0):
    goal.x = gr_points[0][0]
    goal.y =480 -gr_points[0][1]

print('while start')
while not (stop_video):#(George.get_pos() != George.get_goal()):
    
    
    # Reupere la position et angle --> XAV
    # Current goal update --> ANTOINE
    # Check obstacle --> ALICIA
    # Faire avancer --> NOUR

    
    
    x_robot, y_robot,angle_robot, stop_video = update(VideoCap)
    print (x_robot, y_robot,angle_robot)
    x_goal , y_goal = get_goal()
    print (x_goal, y_goal)
    proximity = proxi()
        #LOCAL NAVIGATION
    if move:
            #Thymio is avoiding obstacles
        move = avoid_obstacle(prox_horizonta=proximity) 
    else:
            #Thymio is checking if there's an obstacle in front of it
        speed_l, speed_r = move_to_position(x_robot, y_robot, angle_robot, x_goal, y_goal)
        print (speed_l, speed_r)
        motors(speed_l, speed_r)        
        move = check_cars(prox_horizonta=proximity)
            #FUNCTION NOUR --> OPTIMAL PATH

        #sleep(0.5)

    #led_arrivé
    print(move)

In [ ]:
motors(0,0)